In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install transformers

Mounted at /content/drive
     |████████████████████████████████| 4.0 MB 7.6 MB/s 
     |████████████████████████████████| 6.6 MB 54.2 MB/s 
     |████████████████████████████████| 77 kB 7.5 MB/s 
     |████████████████████████████████| 880 kB 42.3 MB/s 
     |████████████████████████████████| 596 kB 58.8 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=8da10d9292cf37669d5824f662132f0552bf595692f65e99d7a98e5f07002344
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import torch
import re
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import XLMRobertaTokenizerFast, XLMRobertaForSequenceClassification, Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer, RobertaTokenizerFast
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.preprocessing import LabelEncoder
import numpy as np
import random
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available

In [ ]:
model_path = "/content/drive/MyDrive/clef_data/models/1A_en"
test_path = '/content/drive/MyDrive/clef_data/tests/CT22_english_1A_checkworthy_test.tsv'
result_path = '/content/drive/MyDrive/clef_data/tests/subtask1A_checkworthy_english.tsv'
max_length = 64
saved_model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2)
saved_tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

# Push model to device
saved_model.to(device)

# Model to evaluation mode
saved_model.eval()

XLMRobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
     

In [ ]:
def get_prediction(text):
    text = nltk_preprocess(text)
    # prepare our text into tokenized sequence
    inputs = saved_tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors="pt")
    # perform inference to our model
    outputs = saved_model(**inputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1)

    return int(probs.argmax())

In [ ]:
def clean_text(text):
    text = str(text).replace(r'http[\w:\/\.]+', '@link')
    text = str(text).replace(r'\s\s+', ' ')
    return text

def nltk_preprocess(text):
    text = clean_text(text)
    return  text

In [ ]:
test_df = pd.read_csv(test_path, sep='\t')
grade = []
modelId = []
for index, row in test_df.iterrows():
    cur = get_prediction(row['tweet_text'])
    grade.append(cur)
    modelId.append("RoBERTa-large")
    print(row['tweet_text'], cur)
test_df = test_df.drop(['tweet_url', 'tweet_text'],axis=1)
test_df['class_label'] = grade
test_df['run_id'] = modelId
print(test_df.head())
test_df.to_csv(result_path, index=False, sep="\t")